   # Within-fire variability 
   
   Workflow to extract pixels from imagery and boostrap samples to get median and mean for each image.

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import dask
%matplotlib inline
%tb


In [214]:
# required functions
%run processing_functions.py
%run ../pixel_to_df.py

<Figure size 640x480 with 0 Axes>

## Test workflow on single fire

What is the distribution of pixel counts in  defol/non-defoliated between and within fires?


Goals: 
1) create reference distribution from one of defoliation / non-defoiated halves
2) from the opposite half, subsample to match sample size of reference. 
3) boostrap resample to get CI and distrbution of values. 
4) compare distribution

Steps:
1) get coordinates and return them as a dataframe with x and y
2) add column to df which says whether it is defoliated or non-defoliated (binary)
3) extract pixel values? 
4) randomly select a subsample of pixels from df, get values...etc.?

* turn into 4d array with time, band, x, y - band would be nbr and defoliated (1,0)


NOTES:
- values to be standardized when in DF format after pixel extraction

In [134]:
# read in data and get coordinates
ds = rx.open_rasterio("/Users/jgoldman/Desktop/chp3/clipped/nbr/non-defoliated/COC16_1997_1038.nc", masked=True)


In [243]:
# random sample for a given year.
# for each sample take the coordinates and get values across df.
def random_sample(da,
                  n, 
                  dim_slice,
                  keep_cols=True):
    """
    take a data array, turn to df and get random sample
    da = xarray data array with n dimensions
    n = number of samples
    dim_slice = slice along certain time dimension
    keep_cols = 

    """
    # slice along dimension
    da2= da.isel(time=dim_slice)
    #to dataframe
    df = da2.to_dataframe()

    # sample location
    samples = []
    no_of_points = n
    # random sample of n points
    sample_loc = df.dropna().sample(n=int(round(no_of_points)))
    samples.append(sample_loc)
    #join back into single datafame
    all_samples = pd.concat([samples[i] for i in range(0,len(samples))])
    
    # x and y to lat and lon column
    all_samples.reset_index(inplace = True)
    all_samples = all_samples.rename(columns={'y': 'Lat', 'x': 'Long'}) 
    
    if keep_cols==False:
        #remove crs, nbr and time cols
        all_samples = all_samples.drop(columns =['crs', 'time', 'nbr'])
     # for each lat/long select pixel
    pix_samples = []
    for index, row in all_samples.iterrows():
        x = row['Long']
        y = row['Lat']
        df2 = da.to_dataframe()
        df2 = df2.dropna()
        df2 = std_nbr(df2)
        df2 = df2.loc[(df2['y'] == y) & (df2['x'] == x)]
        # add add row that goes preNBR - nbr 10
        ints = np.arange(0,12,1)
        df2['nbr_time'] = ints
        df2['nbr_time'] = df2['nbr_time'].replace({'0': 'preNBR', '1': 'nbrYOF', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8': '7', '9':'8', '10': '9', '11': '10'})
        pix_samples.append(df2)

    #join back into single datafame
    all_samples = pd.concat([pix_samples[i] for i in range(0,len(pix_samples))])
  
    #remove crs
    all_samples = all_samples.drop(columns =['crs'])
    
    ids = np.arange(1,1001,1)
    # add pixel id
    all_samples['id'] = np.repeat(ids, 12, axis = 0)
    
    return(all_samples)

ERRORS IN /1000 (std_nbr) at row nbr_time 1

In [244]:
df = random_sample(ds, 1000, 1, False)
df



,year,y,x,nbr,nbr_time,id
18327,1996,49.890589,-80.574165,0.529663,0,1
48509,1997,49.890589,-80.574165,0.031447,1,1
78691,1998,49.890589,-80.574165,0.093178,2,1
108873,1999,49.890589,-80.574165,0.154908,3,1
139055,2000,49.890589,-80.574165,0.216639,4,1
...,...,...,...,...,...,...
236419,2003,49.873611,-80.504905,0.368056,7,1000
266601,2004,49.873611,-80.504905,0.435173,8,1000
296783,2005,49.873611,-80.504905,0.502290,9,1000
326965,2006,49.873611,-80.504905,0.506911,10,1000


In [309]:
df.iloc[48:60, 0:10]

,year,y,x,nbr,nbr_time,id
26070,1996,49.871994,-80.557726,0.693009,0,5
56252,1997,49.871994,-80.557726,-174.854721,1,5
86434,1998,49.871994,-80.557726,-0.042279,2,5
116616,1999,49.871994,-80.557726,0.090298,3,5
146798,2000,49.871994,-80.557726,0.222874,4,5
176980,2001,49.871994,-80.557726,0.355450,5,5
207162,2002,49.871994,-80.557726,0.488026,6,5
237344,2003,49.871994,-80.557726,0.449980,7,5
267526,2004,49.871994,-80.557726,0.411935,8,5
297708,2005,49.871994,-80.557726,0.373889,9,5


In [303]:
-34.753365 / 1000

-0.034753365

In [296]:
def magnitude_calc(x):
    pre= x[x['nbr_time'] == 0]
    pre = pre['nbr'].item()
    post= x[x['nbr_time'] == 1]
    post = post['nbr'].item()
    p_diff = pre - post
    rec = x[x['nbr_time'] == 11]
    rec = rec['nbr'].item()
    rec_diff = rec - post
    mag = (rec - post)* 100
    
    return pd.DataFrame({'recovery_magnitude': [mag]})



In [307]:
def recovery_magnitude(df):
    grp = df.groupby('id')
    magnitudes = []
    # for length in 1-1001
    ids = np.arange(1,1001,1)
    for i in ids:
        # get group
        x = grp.get_group(i)
        # calc mac
        res = magnitude_calc(x)
        # add id column
        res['id'] = i
        # append to list
        magnitudes.append(res)
        
        
    #join back into single datafame
    all_mags = pd.concat([magnitudes[i] for i in range(0,len(magnitudes))])
    
    return(all_mags)


In [308]:
recovery_magnitude(df)

,recovery_magnitude,id
0,14.283634,1
0,3497.312792,2
0,19.484085,3
0,21.909201,4
0,17526.519063,5
...,...,...
0,921.237145,996
0,-1.984182,997
0,26092.807323,998
0,20.877206,999
